<a href="https://colab.research.google.com/github/prasanna-emperor/xyz.seasonal/blob/main/Copy_of_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch

In [ ]:
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [ ]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [ ]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.8642, -0.5115, -0.9874],
        [-1.3473,  1.0618, -1.1670]], requires_grad=True)
tensor([-0.0754,  0.7414], requires_grad=True)


In [ ]:
def model(x):
    return x @ w.t() + b

In [ ]:
preds = model(inputs)
print(preds)

tensor([[-139.8885,  -76.6519],
        [-186.9204, -103.1120],
        [-201.0684,  -41.8778],
        [-146.7493, -134.2062],
        [-177.9248,  -71.9779]], grad_fn=<AddBackward0>)


In [ ]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [ ]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()


# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(47952.7422, grad_fn=<DivBackward0>)


In [ ]:
loss.backward()


# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.8642, -0.5115, -0.9874],
        [-1.3473,  1.0618, -1.1670]], requires_grad=True)
tensor([[-20624.5645, -22763.1387, -14008.5098],
        [-15026.9307, -15383.3809,  -9843.0391]])


w
w.grad



In [ ]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [ ]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)





tensor(47952.7422, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the .zero_() method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke .backward on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [ ]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


Train the model using gradient descent
As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can train the model using the following steps:

Generate predictions

Calculate the loss

Compute gradients w.r.t the weights and biases

Adjust the weights by subtracting a small quantity proportional to the gradient

Reset the gradients to zero

Let's implement the above step by step.

In [ ]:
preds = model(inputs)
print(preds)

# Calculate the loss
loss = mse(preds, targets)
print(loss)

# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)


# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()


print(w)
print(b)

tensor([[-103.5552,  -51.1411],
        [-139.1526,  -69.5988],
        [-144.4950,   -2.4799],
        [-110.7385, -108.6202],
        [-132.0328,  -39.9494]], grad_fn=<AddBackward0>)
tensor(32481.8594, grad_fn=<DivBackward0>)
tensor([[-16872.4141, -18728.5742, -11519.3945],
        [-12395.0605, -12560.8506,  -8099.9238]])
tensor([-202.1948, -146.3578])
tensor([[-0.4892, -0.0966, -0.7321],
        [-1.0731,  1.3412, -0.9875]], requires_grad=True)
tensor([-0.0709,  0.7447], requires_grad=True)


In [ ]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(22054.7930, grad_fn=<DivBackward0>)


In [ ]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)


tensor(246.2386, grad_fn=<DivBackward0>)


In [ ]:
preds

tensor([[ 60.5958,  68.4439],
        [ 78.5478,  87.1824],
        [121.4996, 166.6346],
        [ 40.4802,  24.2105],
        [ 84.2219, 103.2946]], grad_fn=<AddBackward0>)

**Linear regression using PyTorch built-ins**

In [ ]:
import torch.nn as nn
#As before, we represent the inputs and targets and matrices.

# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [ ]:
from torch.utils.data import TensorDataset
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [ ]:
from torch.utils.data import DataLoader
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
#We can use the data loader in a for loop. Let's look at an example.

for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [ 88., 134.,  59.],
        [ 87., 135.,  57.],
        [102.,  43.,  37.]])
tensor([[119., 133.],
        [ 56.,  70.],
        [118., 132.],
        [118., 134.],
        [ 22.,  37.]])


In [ ]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)


Parameter containing:
tensor([[ 0.1483, -0.0709,  0.1577],
        [ 0.2443, -0.0641, -0.1299]], requires_grad=True)
Parameter containing:
tensor([0.4198, 0.4135], requires_grad=True)


In [ ]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1483, -0.0709,  0.1577],
         [ 0.2443, -0.0641, -0.1299]], requires_grad=True),
 Parameter containing:
 tensor([0.4198, 0.4135], requires_grad=True)]

In [ ]:
preds = model(inputs)
preds

tensor([[13.2706,  8.3666],
        [17.7608,  8.6899],
        [12.9584,  5.5448],
        [18.3274, 17.7678],
        [14.8770,  2.0238],
        [13.4899,  8.6749],
        [17.9894,  8.6241],
        [13.2643,  5.6591],
        [18.1082, 17.4594],
        [14.8864,  1.6496],
        [13.4992,  8.3007],
        [17.9800,  8.9983],
        [12.7298,  5.6106],
        [18.3181, 18.1420],
        [14.6578,  1.7155]], grad_fn=<AddmmBackward0>)

In [ ]:
# Import nn.functional
import torch.nn.functional as F
#The nn.functional package contains many useful loss functions and several other utilities.

# Define loss function
loss_fn = F.mse_loss
#Let's compute the loss for the current predictions of our model.

loss = loss_fn(model(inputs), targets)
print(loss)


tensor(6731.4468, grad_fn=<MseLossBackward0>)


In [ ]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [ ]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):

    # Repeat for given number of epochs
    for epoch in range(num_epochs):

        # Train with batches of data
        for xb,yb in train_dl:

            # 1. Generate predictions
            pred = model(xb)

            # 2. Calculate loss
            loss = loss_fn(pred, yb)

            # 3. Compute gradients
            loss.backward()

            # 4. Update parameters using gradients
            opt.step()

            # 5. Reset the gradients to zero
            opt.zero_grad()

        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))# Utility function to train the model


Some things to note above:

We use the data loader defined earlier to get batches of data for every iteration.

Instead of updating parameters (weights and biases) manually, we use opt.step to perform the update and opt.zero_grad to reset the gradients to zero.

We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. loss.item returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [ ]:
fit(100, model, loss_fn, opt, train_dl)
"""Epoch [10/100], Loss: 818.6476
Epoch [20/100], Loss: 335.3347
Epoch [30/100], Loss: 190.3544
Epoch [40/100], Loss: 131.6701
Epoch [50/100], Loss: 77.0783
Epoch [60/100], Loss: 151.5671
Epoch [70/100], Loss: 151.0817
Epoch [80/100], Loss: 67.6262
Epoch [90/100], Loss: 53.6205
Epoch [100/100], Loss: 33.4517
Let's generate predictions using our model and verify that they're close to our targets."""

# Generate predictions
preds = model(inputs)
preds

Epoch [10/100], Loss: 624.6790
Epoch [20/100], Loss: 257.4446
Epoch [30/100], Loss: 204.8546
Epoch [40/100], Loss: 166.6882
Epoch [50/100], Loss: 130.0726
Epoch [60/100], Loss: 149.0137
Epoch [70/100], Loss: 91.4227
Epoch [80/100], Loss: 68.9246
Epoch [90/100], Loss: 58.1618
Epoch [100/100], Loss: 46.5204


tensor([[ 58.3655,  71.9974],
        [ 81.4972,  97.6476],
        [117.1161, 136.4663],
        [ 28.5005,  46.5252],
        [ 96.5302, 108.4588],
        [ 57.2978,  71.0458],
        [ 81.1779,  97.1452],
        [117.3643, 136.7899],
        [ 29.5682,  47.4769],
        [ 97.2787, 108.9081],
        [ 58.0463,  71.4950],
        [ 80.4295,  96.6959],
        [117.4353, 136.9687],
        [ 27.7520,  46.0759],
        [ 97.5979, 109.4104]], grad_fn=<AddmmBackward0>)

In [ ]:
targets
"""tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])
Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input."""

model(torch.tensor([[75, 63, 44.]]))
#tensor([[55.3323, 67.8895]], grad_fn=<AddmmBackward>)

tensor([[55.0933, 68.7029]], grad_fn=<AddmmBackward0>)